<a href="https://colab.research.google.com/github/AlbertVong/Deep-Learning-Notes/blob/main/active_learning_materials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a loosely copied notebook from Nanohub performing active learning on a small materials dataset. This is to prepare for the materials hackathon on January 2023. 

In [ ]:
#Need this for matminer, importing data from Citrine
!pip install pymatgen
!pip install matminer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Need this for citrination client
!pip install --upgrade citrination-client

In [ ]:
#Lolopy
!pip install lolopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 20.6 MB/s eta 0:00:00


In [ ]:
%matplotlib inline
#Plotting
from matplotlib import pyplot as plt
import matplotlib.animation as animation




In [ ]:
#Matminer/ Citrination
from matminer.data_retrieval.retrieve_Citrine import CitrineDataRetrieval
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from lolopy.learners import RandomForestRegressor

In [ ]:
#PYTHON
import pandas as pd
import numpy as np
np.random.seed(1)
import os
from pymatgen.core import Composition
from scipy.stats import norm
from IPython.display import HTML



ImportError: ignored

In [ ]:
#Citirne api key
apikey = 'QgRN4AV34XELWjFc07boYQtt'

In [ ]:
cdr = CitrineDataRetrieval(apikey)

In [ ]:
#Getting a dataset
data = cdr.get_dataframe(criteria={'data_set_id': 184738}, print_properties_options = False)
property_interest = 'Bulk modulus' #Query property, bulk modulus
data = data.dropna()
data = data[~data.chemicalFormula.str.contains("Ne")]



  0%|          | 0/919 [00:00<?, ?it/s]/usr/local/lib/python3.7/site-packages/matminer/data_retrieval/retrieve_Citrine.py:121: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  system_normdf = json_normalize(system_value)
100%|██████████| 919/919 [00:06<00:00, 144.27it/s]


In [ ]:
#Displaying data
display(data.head(n=10))

,chemicalFormula,Bulk modulus,Bulk modulus-units
1,Hf3Sb,114,GPa
2,PtN2,120,GPa
3,Be2B,150,GPa
4,BaS,41,GPa
5,TePt,55,GPa
6,LiZnPS4,31,GPa
7,VP,179,GPa
8,La3Si2,58,GPa
9,Mg2Si,53,GPa
10,CuNi3N,196,GPa


In [ ]:
Composition??

In [ ]:
#Get compositions using pymatgen
def get_composition(c):
  try:
    return Composition(c)
  except:
    return None

In [ ]:
data['composition'] = data['chemicalFormula'].apply(get_composition)
data[property_interest] = pd.to_numeric(data[property_interest], errors = 'coerce')
display(data.head(n=10))

,chemicalFormula,Bulk modulus,Bulk modulus-units,composition
1,Hf3Sb,114,GPa,"(Hf, Sb)"
2,PtN2,120,GPa,"(Pt, N)"
3,Be2B,150,GPa,"(Be, B)"
4,BaS,41,GPa,"(Ba, S)"
5,TePt,55,GPa,"(Te, Pt)"
6,LiZnPS4,31,GPa,"(Li, Zn, P, S)"
7,VP,179,GPa,"(V, P)"
8,La3Si2,58,GPa,"(La, Si)"
9,Mg2Si,53,GPa,"(Mg, Si)"
10,CuNi3N,196,GPa,"(Cu, Ni, N)"


In [ ]:
data.reset_index(drop=True, inplace=True)
display(data.head(n=10))

,chemicalFormula,Bulk modulus,Bulk modulus-units,composition
0,Hf3Sb,114,GPa,"(Hf, Sb)"
1,PtN2,120,GPa,"(Pt, N)"
2,Be2B,150,GPa,"(Be, B)"
3,BaS,41,GPa,"(Ba, S)"
4,TePt,55,GPa,"(Te, Pt)"
5,LiZnPS4,31,GPa,"(Li, Zn, P, S)"
6,VP,179,GPa,"(V, P)"
7,La3Si2,58,GPa,"(La, Si)"
8,Mg2Si,53,GPa,"(Mg, Si)"
9,CuNi3N,196,GPa,"(Cu, Ni, N)"


In [ ]:
len(data)

918

In [ ]:
#Featurizer, learn how this works later
f = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                        cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])

#Inputs
X = np.array(f.featurize_many(data['composition']))

#Labels - modulus
y = data[property_interest].values

MultipleFeaturizer:   0%|          | 0/918 [00:00<?, ?it/s]

In [ ]:
X.shape

(918, 145)

In [ ]:
#We're gonna initialize rng seed
np.random.seed(44)

in_train = np.zeros(len(data), dtype = np.bool)
#Pick 10 different indices from the number of individual materials (918))
#Turn those to true
in_train[np.random.choice(len(data), 10, replace=False)] = True

#Print
print(f'Picked {in_train.sum()} training entries')

#Make sure you don't pick the max result immediately (or else that defeats the purpose)
assert not np.isclose(max(y), max(y[in_train]))

Picked 10 training entries


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [ ]:
#List of picked "materials/input indices"
np.where(in_train)

(array([ 19,  85, 137, 438, 476, 558, 600, 613, 657, 671]),)

In [ ]:
#Going to use random forest as regression tool.

model = RandomForestRegressor()
model.fit(X[in_train], y[in_train])
y_pred, y_std = model.predict(X[~in_train], return_std=True)

# Active Learning

In [ ]:
mei_selection = np.argmax(y_pred)
mli_selection = np.argmax(np.divide(y_pred - np.argmax(y[in_train]), y_std))
mu_selection = np.argmax(y_std)

print(f'Bulk modulus of material {mei_selection} selected based on MEI: {y_pred[mei_selection]:.2f} +/- {y_std[mei_selection]:.2f}')
print(f'Bulk modulus of material {mli_selection} selected based on MEI: {y_pred[mli_selection]:.2f} +/- {y_std[mli_selection]:.2f}')
print(f'Bulk modulus of material {mu_selection} selected based on MEI: {y_pred[mu_selection]:.2f} +/- {y_std[mu_selection]:.2f}')

Bulk modulus of material 358 selected based on MEI: 239.40 +/- 34.26
Bulk modulus of material 130 selected based on MEI: 234.90 +/- 32.42
Bulk modulus of material 319 selected based on MEI: 170.60 +/- 119.23


We will run 100 experiments to see if we can reach the max value of bulk modulus candidates in Citrination. Each block contains steps of training/predicting/adding posterior samples to each strategy

In [ ]:
n_steps = 100
all_inds = set(range(len(y)))

#Create the initial training set for all strategies
random_train = [list(set(np.where(in_train)[0].tolist()))] #benchmarked random strat
mei_train = [list(set(np.where(in_train)[0].tolist()))]
mli_train = [list(set(np.where(in_train)[0].tolist()))]
mu_train = [list(set(np.where(in_train)[0].tolist()))]

random_train_inds = []
mei_train_inds = []
mli_train_inds = []
mu_train_inds = []

In [ ]:
for i in range(n_steps):

  #Random
  random_train_inds = random_train[-1].copy() #Initial set
  random_search_inds = list(all_inds.difference(random_train_inds)) #Which indices am I searching through?
  model.fit(X[random_train_inds], y[random_train_inds]) #Fit everything else that wasn't trained on
  random_train_inds.append(np.random.choice(random_search_inds)) #Randomly select an option to search next
  random_train.append(random_train_inds)

  #Maximum expected improvement
  mei_train_inds = mei_train[-1].copy()
  mei_search_inds = list(all_inds.difference(mei_train_inds))
  model.fit(X[mei_train_inds], y[mei_train_inds])
  mei_y_pred = model.predict(X[mei_search_inds])
  mei_train_inds.append(mei_search_inds[np.argmax(mei_y_pred)])

  #Maximum likelihood
  mli_train_inds = mli_train[-1].copy()  # Last iteration
  mli_search_inds = list(all_inds.difference(mli_train_inds))
  model.fit(X[mli_train_inds], y[mli_train_inds])
  mli_y_pred, mli_y_std = model.predict(X[mli_search_inds], return_std=True)
  mli_train_inds.append(mli_search_inds[np.argmax(np.divide(mli_y_pred - np.max(y[mli_train_inds]), mli_y_std))])     # Pick entry with the largest MLI value
  mli_train.append(mli_train_inds)

  #Maximum uncertainty
  mu_train_inds = mu_train[-1].copy()  # Last iteration
  mu_search_inds = list(all_inds.difference(mu_train_inds))
  model.fit(X[mu_train_inds], y[mu_train_inds])
  mu_y_pred, mu_y_std = model.predict(X[mu_search_inds], return_std=True)
  mu_train_inds.append(mu_search_inds[np.argmax(mu_y_std)])     # Pick entry with the largest uncertainty
  mu_train.append(mu_train_inds)

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(30,8))

#FIRST PLOT

random_line, = ax[0].plot([],[], color='green', label='Random')
mei_line, = ax[0].plot([],[],color='blue', label='MEI')
mli_line, = ax[0].plot([], [], color='red', label='MLI')
mu_line, = ax[0].plot([], [], color='purple', label='MU')
#Just a long line showing the max value across all steps
max_line, = ax[0].plot(range(n_steps), [max(y) for m in range(n_steps)], '--', color='black', label='Max Value')

random_chk, = ax[0].plot([],[], markersize=10, marker='*', linestyle='None', color='green')
mei_chk, = ax[0].plot([],[], markersize=10, marker='*', linestyle='None', color='blue')
mli_chk, = ax[0].plot([],[], markersize=10, marker='*', linestyle='None', color='red')
mu_chk, = ax[0].plot([],[], markersize=10, marker='*', linestyle='None', color='purple')

ax[0].legend(loc=4, prop={'size': 12})
ax[0].grid()
ax[0].set_title("Number of Experiments", fontsize=24)
ax[0].set_xlabel("Number of Experiments", fontsize=16)
ax[0].set_ylabel("Max Bulk Modulus", fontsize=16)
mli_line.axes.axis([0, n_steps-1, 0, 1.1*max(y)])
mli_line.axes.get_yaxis().set_tick_params(labelsize=14)

#Second Plot, Random prediction
#Plot x, y values for all test sets
all_values_samples = ax[1].plot(list(all_inds), y, marker='o', alpha=0.2, color = 'gray', linestyle = 'None', markersize=10, label = 'Values')
#Y values for all the original training set
random_reallabel = [y[index] for index in random_train_inds]
#Plot x, y values for training set in dark circles
random_initial_set = ax[1].plot(random_train_inds[:10], random_reallabel[:10], color='black', marker='o', linestyle = 'None', markersize=10, label = 'Initial Set')

#Plot test points (empty for now)
random_sample_real, = ax[1].plot([], [], color='green', marker='o', linestyle= 'None',  markersize=10, label = 'Tests')
random_sample_real.axes.axis([0, len(y), 0, 1.1*max(y)])
random_sample_real.axes.get_xaxis().set_ticks([])
random_sample_real.axes.get_yaxis().set_tick_params(labelsize=14)

ax[1].legend(bbox_to_anchor=(0.8, 0.1), prop={'size': 12})
ax[1].grid()
ax[1].set_title("Random", fontsize=24)
ax[1].set_xlabel("Test Candidates", fontsize=16)
ax[1].set_ylabel("Bulk Modulus", fontsize=16)

#Third set of plots
all_values_samples = ax[2].plot(list(all_inds), y, marker='o', alpha=0.2, color='gray', linestyle='None', markersize=10, label='Values')

mei_reallabel = [y[index] for index in mei_train_inds]

mei_initial_set = ax[2].plot(mei_train_inds[:10], mei_reallabel[:10], color='black', marker='o',linestyle= 'None',   markersize=10, label = 'Initial Set')

mei_sample_real, = ax[2].plot([], [], color='blue', marker='o', linestyle= 'None',  markersize=10, label = 'Tests')
mei_sample_real.axes.axis([0, len(y), 0, 1.1*max(y)])
mei_sample_real.axes.get_xaxis().set_ticks([])
mei_sample_real.axes.get_yaxis().set_tick_params(labelsize=14)


ax[2].legend(bbox_to_anchor=(0.8, 0.1), prop={'size': 12})
ax[2].grid()
ax[2].set_title("MEI", fontsize=24)
ax[2].set_xlabel("Test Candidates", fontsize=16)
ax[2].set_ylabel("Bulk Modulus", fontsize=16)

# 4th PLOT, MLI Prediction

all_values_samples = ax[3].plot(list(all_inds), y, marker='o', alpha=0.2, color='gray', linestyle='None', markersize=10, label='Values')

mli_reallabel = [y[index] for index in mli_train_inds]

mli_initial_set = ax[3].plot(mli_train_inds[:10], mli_reallabel[:10], color='black', marker='o', linestyle= 'None',  markersize=10, label = 'Initial Set')
mli_sample_real, = ax[3].plot([], [], color='red', marker='o', linestyle= 'None',  markersize=10, label = 'Tests')
mli_sample_real.axes.axis([0, len(y), 0, 1.1*max(y)])
mli_sample_real.axes.get_xaxis().set_ticks([])
mli_sample_real.axes.get_yaxis().set_tick_params(labelsize=14)


ax[3].legend(bbox_to_anchor=(0.8, 0.1), prop={'size': 12})
ax[3].grid()
ax[3].set_title("MLI", fontsize=24)
ax[3].set_xlabel("Test Candidates", fontsize=16)
ax[3].set_ylabel("Bulk Modulus", fontsize=16)


# 5th plot, MU Prediction

all_values_samples = ax[4].plot(list(all_inds), y, marker='o', alpha=0.2, color='gray', linestyle='None', markersize=10, label='Values')

mu_reallabel = [y[index] for index in mu_train_inds]

mu_initial_set = ax[4].plot(mu_train_inds[:10], mu_reallabel[:10], color='black', marker='o', linestyle= 'None',  markersize=10, label = 'Initial Set')
mu_sample_real, = ax[4].plot([], [], color='purple', marker='o', linestyle= 'None',  markersize=10, label = 'Tests')
mu_sample_real.axes.axis([0, len(y), 0, 1.1*max(y)])
mu_sample_real.axes.get_xaxis().set_ticks([])
mu_sample_real.axes.get_yaxis().set_tick_params(labelsize=14)


ax[4].legend(bbox_to_anchor=(0.8, 0.1), prop={'size': 12})
ax[4].grid()
ax[4].set_title("MU", fontsize=24)
ax[4].set_xlabel("Test Candidates", fontsize=16)
ax[4].set_ylabel("Bulk Modulus", fontsize=16)

##### ANIMATION #####

def update(num):
    
    if num > 0:
        
        random_graph = [max(y[list(t)]) for t in random_train[:num]]
        if max(y) in random_graph:
            chk_index = [i for i, j in enumerate(random_graph) if j == max(y)][0]
            random_line.set_data(np.arange(len(random_train))[:chk_index+1], [max(y[list(t)]) for t in random_train[:chk_index+1]])
            random_chk.set_data(chk_index, max(random_graph))
        else:
            random_line.set_data(np.arange(len(random_train))[:num], random_graph)
            random_sample_real.set_data(random_train_inds[10:10+num], random_reallabel[10:10+num])
        
        mei_graph = [max(y[list(t)]) for t in mei_train[:num]]
        if max(y) in mei_graph:
            chk_index = [i for i, j in enumerate(mei_graph) if j == max(y)][0]
            mei_line.set_data(np.arange(len(mei_train))[:chk_index+1], [max(y[list(t)]) for t in mei_train[:chk_index+1]])
            mei_chk.set_data(chk_index, max(mei_graph))
        else:
            mei_line.set_data(np.arange(len(mei_train))[:num], mei_graph)
            mei_sample_real.set_data(mei_train_inds[10:10+num], mei_reallabel[10:10+num])
        
        mli_graph = [max(y[list(t)]) for t in mli_train[:num]]
        if max(y) in mli_graph:
            chk_index = [i for i, j in enumerate(mli_graph) if j == max(y)][0]
            mli_line.set_data(np.arange(len(mli_train))[:chk_index+1], [max(y[list(t)]) for t in mli_train[:chk_index+1]])
            mli_chk.set_data(chk_index, max(mli_graph))
        else:
            mli_line.set_data(np.arange(len(mli_train))[:num], mli_graph)
            mli_sample_real.set_data(mli_train_inds[10:10+num], mli_reallabel[10:10+num])          
            
        mu_graph = [max(y[list(t)]) for t in mu_train[:num]]
        if max(y) in mu_graph:
            chk_index = [i for i, j in enumerate(mu_graph) if j == max(y)][0]
            mu_line.set_data(np.arange(len(mu_train))[:chk_index+1], [max(y[list(t)]) for t in mu_train[:chk_index+1]])
            mu_chk.set_data(chk_index, max(mu_graph))
        else:
            mu_line.set_data(np.arange(len(mu_train))[:num], mu_graph)
            mu_sample_real.set_data(mu_train_inds[10:10+num], mu_reallabel[10:10+num])   
            
        return random_line, mei_line, mli_line, mu_line, random_sample_real, mei_sample_real,mli_sample_real, mu_sample_real , mli_chk, mei_chk, random_chk, mu_chk

anim = animation.FuncAnimation(fig, update, frames=len(random_train), interval=600, blit=False, repeat = False)


plt.close(fig)

HTML(anim.to_html5_video())